In [4]:
import pandas as pd
import numpy as np
import polars as pl

import plotly.express as px

import pydsptools.config.pda as pda
import pydsptools.biorad as biorad

import pydsp.run.worker
import pydsptools.biorad.parse as bioradparse
import pydsptools.plot as dspplt
import pprint
import pyarrow as pa
import os
import subprocess
import matplotlib.pyplot as plt
from pathlib import Path

from pydsptools.biorad import DataParser  # BioRad PCR data parser
from pydsptools.config import ConfigParser  # Config parser
from pydsptools.datalake import Dataset # Data storage
from pydsptools.analysis import CompareResults  # Analysis

# 담당자 

* 프로젝트 발의자: 손형석 차장
* 프로젝트 담당자: 김광민 대리
* 문서 작성자: 김광민 대리 

# Project Background

[2024-02-05] 구체적인 목표 및 action items이 부재한 추상적인 상태이다. 
* baseline fitting 알고리즘이 CFX manager, DSP-Legacy, and AutoBaseline 등과 같이 여러 종류가 있고 그 fitting 성능이 직관적으로 보기에 개선의 필요성을 느낌.
* 이 프로젝트를 통해서 각 각의 baseline 알고리즘의 장/단점을 도출하고 더 향상된 baseline fitting알고리즘을 개발하는 것을 이 프로젝트의 목적으로 한다.

# Goals

[2024-02-06] 다음과 같은 일시적인 목표를 정함.

* insights 도출과 직관적인 분석을 위해 3개의 알고리즘의 fitting 결과를 시각화한다. 
* 시각화에 필요한 분석 환경을 마련한다.
    * 기제품 raw data 확보, 전처리하여 dsp 연산
    * 기제품 data baseline-subtracted by cfx manager 확보, 전처리하여 dsp 연산
    * 전처리 auto-baseline 처리된 data 확보

[2024-04-19] 더 구체화된 목표와 action items이 생성됨 (자세한 기록은 팀 주간보고 onenote 참고)

1. baseline fitting 알고리즘 비교분석을 위한 plate 단위 신호 시각화: plate 당 Channel 당 Temperature당 신호 시각화를 6개 패널로 Fix
    *  [After BPN] RFU: 원본 신호를 BPN (a kind of normalization) 처리된 신호를 출력
    *  [DSP] Original ABSD: DSP 알고리즘의 중간 산출물인 analysis-absd-orig을 출력 (참고: absd = after baseline subtracted data)
    *  [Auto] Baseline-Subtracted RFU: DSP 알고리즘의 모듈인 autobaseline (PGR manager의 Raw tab 결과를 산출하는 모듈)의 결과물 출력
    *  [CFX] Baseline-Subtracted RFU: CFX manager에 의한 baseline subtracted data를 출력
    *  [Strep] Baseline-Subtracted RFU: Strep Assay 제품용 baseline fitting 알고리즘의 결과물 출력 (알고리즘 개발자: 손형석 차장, 알고리즘 구현자: 양우진 부장)
    *  [Strep+n] Baseline-Subtracted RFU: Strep Assay 제품용 baseline fitting 알고리즘의 n번째 개선된 결과물 출력 (알고리즘 개발자: 손형석 차장, 알고리즘 구현자: 양우진 부장)
2. baseline fitting 알고리즘 심층 비교분석을 위한 단일 신호 단위 시각화: plate 당 Channel 당 Temperature당 well당 신호 시각화를 (알고리즘 개수)x(3)개의 패널로 Fix
    * 알고리즘의 개수 = 6개
        * [After BPN] RFU
        * [DSP] Original ABSD
        * [Auto] Baseline-Subtracted RFU
        * [CFX] Baseline-Subtracted RFU
        * [Strep] Baseline-Subtracted RFU
        * [Strep+n] Baseline-Subtracted RFU
3. baseline fitting 알고리즘별 신호 패턴 시각화: 다수의 plate의 걸쳐 channel 당, temperature당, well당 신호를 시각화
    * DSP의 모듈 구조상 선형증가 탈락 신호와 같이 baseline fitting을 거치지 않는 신호는 출력이 불가하기 때문에 시각화에 포함시키지 않음
    * 4개의 패널로 고정
        * [After BPN] RFU
        * [Auto] Baseline-Subtracted RFU
        * [CFX] Baseline-Subtracted RFU
        * [Strep+n] Baseline-Subtracted RFU
    * 신호 pattern 별 시각화 종류 
        * 하향 신호 pattern 시각화
        * 상향 신호 pattern 시각화
        * MuDT 왜곡될 가능성이 있는 신호 시각화 : High에서 양성, Low에서 음성인 well의 신호를 수집  능성이 있는 신호 시각화 : High에서 양성, Low에서 음성인 well의 신호를 수집  

# Script Description

## [2024-02-12]
* 이 스크립트는 실무진들 업무 파악 및 정리가 이루어지기전에 작업한 코드도 섞여 있다.
    * (채택되지 않은 방식): 업무 파악 전 코드 
    * (채택된 방식): 업무 파악 후 코드
* `1.cfx-to-parquet-converter.ipynb` 는 다음 2가지의 data를 전처리한다.
    * CFX Data named  (RFU Baseline Subtracted by CFX Manager) 를 `bioradparse.load_pcrdata()` 함수에 의해 전처리된 결과물인 parquet 파일로 변환한다.
    * Raw Sample Data (sampled from `pda-raw-PRJDS001`) 를 `bioradparse.load_pcrdata()` 함수에 의해 전처리된 결과물인 parquet 파일로 변환한다.
* 최종 결과물인 `merged_data` 는 DSP 연산을 돌리기 전 data를 병합시킨 결과물이다. 

### 1. CFX Data (Baseline Subtracted) Parquet Files 생성

* Seegene Export xlsx or batchanalyzer csv 데이터를 Parquet 파일로 변환하기 위한 path 설정

In [2]:
root_path = Path.cwd() # /home/kmkim/pda/dsp-research-strep-a/kkm
prefix = 'data'
directory_names = ['GI-B-I']#['cfx-baseline-subtracted','pda-raw-sample']
product_names = ['GI-B-I', 'GI-B-II', 'GI-P', 'GI-V', 'RP1', 'RP2', 'RP3', 'RP4', 'STI-CA', 'STI-EA', 'STI-GU']
consumables = ['8-strip','96-cap']
plate_numbers = ['plate_data_' + number for number in ['002','005','031','032','036','041']]
# !python -m pydsptools.biorad.parse -t cfx-batch-csv -f parquet -o './data/baseline-subtracted/processed/example1' './data/baseline-subtracted/cfx-data/'

#### 1) Parquet Files을 `plate_data_{number}` path에 분산하여 실행하는 방식
* (채택되지 않은 방식) : data paths가 복잡해짐
* `bioradparse.load_pcrdata()`를 이용한 batchanalyzer csv 데이터를 Parquet 파일로 변환 (아래 코드 돌릴 필요 없음- 아래의 다른 방식으로 돌릴 예정) 
* 이 방식은 실무진들이 업무 파악 및 정리가 되지 않아 임시 시도한 방식

In [65]:
# cfx_data = []
# raw_data = []
# 
# for directory_name in directory_names: 
#     for product_name in ['GI-B-I']: #product_names:
#         for consumable in ['8-strip']: #consumables:
#             for plate_number in plate_numbers:
#                 full_path = root_path / prefix / directory_name / product_name / consumable / plate_number
#                 processed_path = full_path / "processed" / "example1"
#                 processed_path.mkdir(parents=True, exist_ok=True)
#                 exporting_path =  full_path / "exported_pcrd"
#                 if 'cfx' in exporting_path: 
#                     temp_cfx_data = bioradparse.load_pcrdata(exporting_path, datatype="cfx-xl")
#                     cfx_data.append(temp_cfx_data)
#                 temp_raw_data = bioradparse.load_pcrdata(exporting_path, datatype="cfx-batch-csv")
#                 raw_data.append(temp_raw_data)
# #pathlib.Path(f"./data/baseline-subtracted/processed/example1")
# 
# for pcrname, pcrdata in raw_data.items():
#     bioradparse.save_pcrdata(raw_data, root_path / "pda-raw-sample" / "processed" / "example1" / f"{pcrname}.parquet")
# for pcrname, pcrdata in cfx_data.items():
#     bioradparse.save_pcrdata(cfx_data, root_path / "cfx-baseline-subtraction" / "processed" / "example1" / f"{pcrname}.parquet")

#### 2) Parquet Files을 `./data/cfx-baseline-subtracted/cfx-data` 에 batch로 배치
* (채택된 방식)
* `batchanalyzer.exe`로 CFX Manager Baseline Subtracted Data 대량 batch 추출 후 `directory-path/cfx-data`에 저장  bioradparse module 돌림
* 아래 코드 한번만 돌리면 됨

In [4]:
cfx_datapath = root_path / prefix / directory_names[0] / 'cfx-data'
cfx_data = bioradparse.load_pcrdata(str(cfx_datapath), datatype="cfx-batch-csv") # output: dictionary

In [5]:
len(cfx_data.keys()) # 201개 plates

# Convert PyArrow Tables to DataFrames and store them in a new dictionary
cfx_df_dict = {key: value.to_pandas() for key, value in cfx_data.items()}

# Convert the dictionary of DataFrames to a single DataFrame (concatenating along rows)
cfx_df = pd.concat(cfx_df_dict.values(), axis=0)
#cfx_df = cfx_df.rename(columns={'rfu':'cfx_rfu','endrfu':'cfx_endrfu','melt_idx':'cfx_melt_idx','melt':'cfx_melt'})
cfx_df['combo_key'] = cfx_df.apply(lambda x: f"{x['name']} {x['channel']} {x['step']} {x['well']} {x['welltype']}", axis=1)



In [7]:
# export the preprocessed dataframe 
cfx_df.to_parquet('./data/cfx-baseline-subtracted/merge_cfx-baseline-subtracted_kkm_v1_20240213.parquet', index=False)

### 2. Raw Sample Data 생성
* `/home/kmkim/pda/dsp-research-strep-a/kkm/data/pda-raw-PRJDS001/` 기제품 데이터 중 `GI-B-I/GI-B-I_8-strip` 일부를 sampling함
* sampling 방식은 아래의 criteria를 갖고 manual sampling
    * Incusion criteria: 다음과 같은 조건을 만족시키는 신호 선별 
        * Baseline 차감 전 일정한 pattern 띄는 음성 신호 
        * Baseline 차감 후 음성 신호에서 다른 pattern이 보이는 plate 
        * RFU magnitude 120 이상 
        * Baseline 차감 후 신호의 Main pattern과 다른 pattern 보이는 plate
            * 예) 하향 평행이동한 noise vs V-shape pattern
    * Exclusion criteria: 차감 전 pattern과 차감 후 pattern이 유사한 신호 

In [8]:
for plate_number in plate_numbers:
    directory_path_raw = root_path / prefix / directory_names[1] / product_names[0] / f"{product_names[0]}_{consumables[0]}"
    raw_datapath = directory_path_raw / plate_number
    input_path = str(raw_datapath)+ '/exported_pcrd'
    temp_raw_data = bioradparse.load_pcrdata(input_path, datatype="cfx-batch-csv")
    raw_df_dict = {key: value.to_pandas() for key, value in temp_raw_data.items()}
    raw_df = pd.concat(raw_df_dict.values(), axis=0)
raw_df['combo_key'] = raw_df.apply(lambda x: f"{x['name']} {x['channel']} {x['step']} {x['well']} {x['welltype']}", axis=1)

In [9]:
# export the preprocessed dataframe 
raw_df.to_parquet('./data/pda-raw-sample/merge_pda-raw-sample_kkm_v1_20240213.parquet', index=False)

## [2024-02-12]
### 3. GI-B1 plate_data_001~100 parquet생성

#### 2) Parquet Files을 `./data/GI-B-I/GI-B1_8-strip` 에 batch로 배치

- pydsp tutorial script update로 인한 code 최신화
- 참고: [https://github.com/SeegeneDevelopmentPlatform/pydsp-tutorials/getting-started.ipynb](https://github.com/SeegeneDevelopmentPlatform/pydsp-tutorials/getting-started.ipynb)

**(1) PCR 데이터 준비 (Import)**

DSP 연산을 하고자 하는 PCRD 파일의 Seegene Export 파일을 압축하여 준비한 후, 아래 정보를 입력함

* RAW_ZIP_PATH : PCRD 파일을 Seegene export 파일로 변환하여 압축한 파일의 경로
* TO_PARQUET_DIR : DSP 연산을 위해 Seegene export 파일을 paruqet으로 변환하여 저장할 디렉토리의 경로
* PCR_SYSTEM : 실험에서 사용한 PCR 기기 종류
* CONSUMABLE : 실험에서 사용한 소모품 종류 (8-strip, 96-film, 96-cap 중 선택)
* PROTOCOL : Quantstep 및 Temperature
* EXPERIMENT_NAME : 실험 이름
* PLATE_NAME : 플레이트 디자인 이름
* 위에서 입력한 정보와 함께 아래 코드 cell의 `biorad.DataParser()`를 실행하면 TO_PARQUET_DIR에 PARQUET 파일들이 저장됨

In [5]:
RAW_ZIP_PATH = "./data/GI-B-I/GI-B-I-100/GI-B-I-100.zip"

PCR_SYSTEM = "CFX96"
CONSUMABLE = "8-strip"
PROTOCOL = {4: "Low", 5: "High"} # {QUANTSTEP: TEMPERATURE}
EXPERIMENT_NAME = "MY EXPERIMENT"
PLATE_NAME = "PLATE_NUM_001_100"
TO_PARQUET_DIR = "./data/GI-B-I/GI-B-I-100/computed/pcr_results"

(
    biorad.DataParser(RAW_ZIP_PATH, type="cfx-batch-csv")
    .append_metadata(
        pcr_system=PCR_SYSTEM,
        consumable=CONSUMABLE,
        protocol=PROTOCOL,
        experiment_name=EXPERIMENT_NAME,
        plate_name=PLATE_NAME,
    )
    .dump(TO_PARQUET_DIR)
)

* PCR 파일들의 형태가 PARQUET으로 변환되어 DSP 연산 준비가 된 상태가 되었다는 것을 의미

**(2) 연산 설정 준비 (Prepare the setting file)**

[strep+2] 을 위한 설정값 준비



In [9]:
#CONFIG_NAME = "dsp2_generic_config_MuDT"
#CONFIG_NAME = "dsp2_generic_config_no-MuDT"
#CONFIG_NAME = "dsp2_strep_plus1_config_MuDT"
#CONFIG_NAME = "dsp2_strep_plus1_config_no-MuDT"
#CONFIG_NAME = "dsp2_strep_plus2_config_MuDT"
CONFIG_NAME = "dsp2_strep_plus2_config_no-MuDT"

DSP_VERSION = "2.1.1"
DSP_PRODUCT = "GI-B-I"
COMMENTS = "20240508 Generic Setting Values without MuDT"
CONSUMABLE1 = "8-strip"
CONSUMABLE2 = "96-cap"
CONSUMABLE3 = "96-film"

XSLX_PATH_8_STRIP = "./config/xlsx/기제품/GI-B-I-100/dsp2_strep_plus2_config_no-MuDT_8-strip.xlsx"
XSLX_PATH_96_CAP = "./config/xlsx/기제품/GI-B-I-100/dsp2_strep_plus2_config_no-MuDT_96-cap.xlsx"
XSLX_PATH_96_FILM = "./config/xlsx/기제품/GI-B-I-100/dsp2_strep_plus2_config_no-MuDT_96-film.xlsx"

CONFIG_YML_PATH = "./config/yaml/PRJDS001/GI-B-I/dsp2_strep_plus2_config_no-MuDT.yml"


(
    ConfigParser()
        .set_dsp_version(DSP_VERSION)
        .set_dsp_product(DSP_PRODUCT)
        .set_name(CONFIG_NAME)
        .set_comments(COMMENTS)
        .add_config(CONSUMABLE1, path=XSLX_PATH_8_STRIP)
        .add_config(CONSUMABLE2, path=XSLX_PATH_96_CAP)
        .add_config(CONSUMABLE3, path=XSLX_PATH_96_FILM)
        .dump(CONFIG_YML_PATH)
)


**(3) DSP 연산**

`2.dsp-executer.ipynb` 에서 실행